In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/MyDrive/VRYNT/code/Word_Clustering

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/MyDrive/VRYNT/code/Word_Clustering


In [ ]:
# ! pip install ftfy regex tqdm
# ! pip install git+https://github.com/openai/CLIP.git
! pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os
import skimage
import IPython.display
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

from collections import OrderedDict
import torch

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# Zero-Shot Image Classification

You can classify images using the cosine similarity (times 100) as the logits to the softmax operation.

In [ ]:
from torchvision.datasets import CIFAR100

dataLabels = np.array(CIFAR100(os.path.expanduser("~/.cache"),download=True).classes)

  0%|          | 0/169001437 [00:00<?, ?it/s]

Extracting /root/.cache/cifar-100-python.tar.gz to /root/.cache


In [ ]:
import re
dataLabels = np.array([])
with open('imagenet1000_clsidx_to_labels.txt') as f:
  for line in f:
    word = re.split(''',|'|"|\n''',line)[1]
    dataLabels = np.append(dataLabels,word)    
print(dataLabels[:10])

['tench' 'goldfish' 'great white shark' 'tiger shark' 'hammerhead'
 'electric ray' 'stingray' 'cock' 'hen' 'ostrich']


In [ ]:
import nltk
from nltk.corpus import wordnet as wn
nltk.download('omw-1.4')
nltk.download('wordnet')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
data_synsets = []
data_idxN = []
for i,label in enumerate(dataLabels):
  label_synsets = wn.synsets(label)
  if label_synsets:
    data_synsets.append(label_synsets)
  else:
    labelsplit = re.split('-| ',label)
    label_synsets = wn.synsets("".join(labelsplit))
    if label_synsets:
      data_synsets.append(label_synsets)
    else:
      data_idxN.append(i)

print(len(data_idxN))

401


In [ ]:
import csv
words = np.array([])
with open('gen_string_filled.csv', 'r') as file:
  csvreader = csv.reader(file)
  words = np.append(words,next(csvreader)[0])
  for word in csvreader:
      words = np.append(words,word[0])
      
print('Number of Words:',len(words))
word_synsets = []
word_idxN = []
for i,label in enumerate(words):
  label_synsets = wn.synsets(label)
  if label_synsets:
    word_synsets.append(label_synsets)
  else:
    labelsplit = re.split('-| ',label)
    label_synsets = wn.synsets("".join(labelsplit))
    if label_synsets:
      word_synsets.append(label_synsets)
    else:
      word_idxN.append(i)

print('Number of Unconverted Words:',len(word_idxN))

Number of Words: 6894
Number of Unconverted Words: 254


In [ ]:
word_sim = np.zeros((len(words),len(data_synsets)))
k = 0
for i in range(len(words)):
  if i not in word_idxN:
    for j,data_synset in enumerate(data_synsets):
      max_sim = 0
      for synset in word_synsets[i-k]:
        for dsynset in data_synset:
          sim = synset.wup_similarity(dsynset)
          if sim > max_sim:
            max_sim = sim
          #end if
        #end for
      #end for
      word_sim[i][j] = max_sim
    #end for
  #end if
  else:
      k += 1
      print(i)
  #end else
#end for

17
64
95
102
112
153
154
185
222
227
229
230
231
237
265
394
409
417
464
500
506
577
613
674
692
718
741
760
790
797
937
994
1026
1049
1095
1144
1191
1209
1213
1344
1374
1429
1452
1453
1461
1476
1506
1518
1523
1524
1526
1546
1605
1669
1839
1851
1918
1929
1930
1935
1975
1977
1985
1996
1998
2019
2029
2054
2143
2189
2190
2191
2284
2293
2310
2311
2320
2336
2350
2396
2482
2486
2509
2543
2558
2574
2576
2690
2692
2704
2721
2745
2746
2760
2777
2821
2836
2839
2843
2858
2892
2913
2963
3048
3080
3084
3118
3185
3265
3280
3282
3291
3299
3304
3345
3353
3359
3383
3420
3422
3528
3529
3593
3626
3627
3635
3639
3651
3675
3733
3842
3846
3889
3896
3916
3924
3958
4005
4016
4017
4033
4071
4074
4075
4080
4101
4109
4113
4136
4187
4278
4315
4333
4334
4357
4393
4418
4444
4480
4508
4512
4582
4622
4630
4661
4741
4743
4744
4826
4827
4842
4859
4912
4930
4954
5052
5053
5055
5139
5142
5143
5200
5327
5362
5375
5474
5476
5509
5518
5537
5550
5583
5632
5669
5689
5706
5760
5763
5847
5865
5869
5870
5874
5877
5890
5902
5909


In [ ]:
print(word_idxN)

[17, 64, 95, 102, 112, 153, 154, 185, 222, 227, 229, 230, 231, 237, 265, 394, 409, 417, 464, 500, 506, 577, 613, 674, 692, 718, 741, 760, 790, 797, 937, 994, 1026, 1049, 1095, 1144, 1191, 1209, 1213, 1344, 1374, 1429, 1452, 1453, 1461, 1476, 1506, 1518, 1523, 1524, 1526, 1546, 1605, 1669, 1839, 1851, 1918, 1929, 1930, 1935, 1975, 1977, 1985, 1996, 1998, 2019, 2029, 2054, 2143, 2189, 2190, 2191, 2284, 2293, 2310, 2311, 2320, 2336, 2350, 2396, 2482, 2486, 2509, 2543, 2558, 2574, 2576, 2690, 2692, 2704, 2721, 2745, 2746, 2760, 2777, 2821, 2836, 2839, 2843, 2858, 2892, 2913, 2963, 3048, 3080, 3084, 3118, 3185, 3265, 3280, 3282, 3291, 3299, 3304, 3345, 3353, 3359, 3383, 3420, 3422, 3528, 3529, 3593, 3626, 3627, 3635, 3639, 3651, 3675, 3733, 3842, 3846, 3889, 3896, 3916, 3924, 3958, 4005, 4016, 4017, 4033, 4071, 4074, 4075, 4080, 4101, 4109, 4113, 4136, 4187, 4278, 4315, 4333, 4334, 4357, 4393, 4418, 4444, 4480, 4508, 4512, 4582, 4622, 4630, 4661, 4741, 4743, 4744, 4826, 4827, 4842, 4859, 49

In [ ]:
np.save('./word_sim.npy',word_sim)

In [ ]:
word_sim = np.load('./word_sim.npy')

In [ ]:
cluster_idx = np.argmax(word_sim,axis=1)
# top_probs = np.flip(np.sort(word_sim,axis=-1),axis=-1)[:,:5]
top_labels = np.flip(np.argsort(word_sim,axis=-1),axis=-1)[:,:5]

for idx in word_idxN:
  top_labels[idx] = -1
  cluster_idx[idx] = -1
dataLabels = np.append(dataLabels,'miscellaneous')

In [ ]:
import pandas as pd
df = pd.DataFrame({'Word': words,
                   'ClusterID': cluster_idx,
                   'Cluster_Label': dataLabels[cluster_idx]})
df.to_csv('clusters/wu-palmer_sim_Cluster_ImageNet.csv',index=False)

df = pd.DataFrame({'Word': words,
                   'Top1': dataLabels[top_labels[:,0]],
                   'Top2': dataLabels[top_labels[:,1]],
                   'Top3': dataLabels[top_labels[:,2]],
                   'Top4': dataLabels[top_labels[:,3]],
                   'Top5': dataLabels[top_labels[:,4]]})
df.to_csv('topkWords/wu-palmer_sim_topkWords_ImageNet1000.csv',index=False)